#          Modelo De REGRESSAO LINEAR

# Dados
Vamos começar utilizando como exemplo os dados da competição House Prices: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

In [3]:
# Importanto as bibliotecas

import pandas as pd
import numpy as np
import sklearn, matplotlib

from matplotlib import pyplot as plt

In [11]:
# Carregando os dados
data = pd.read_csv("train.csv")
target = "SalePrice"
data.head()


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [23]:
# Resolvendo o problema de missing 
data.fillna(data.median(), inplace=True)
# Transformando o target no log do valor
data[target] = np.log1p(data[target])
# Excluindo os outliers, pegando as áreas até o valor de 95% do valor máximo
data = data[data["LotArea"] <= data["LotArea"].quantile(0.95)]

In [34]:
## Analisando as correlações
pd.set_option('max_columns', 81)
data.corr().T

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,Pred
Id,1.000000,0.022446,-0.026217,-0.032088,-0.029422,0.015598,-0.030417,-0.032972,-0.057110,-0.010659,-0.005724,-0.018942,-0.034192,-0.008860,0.007566,-0.045436,-0.004132,0.006052,-0.004410,-0.006669,0.004818,0.014178,0.001170,0.011477,-0.012453,-0.016466,0.007726,0.011169,-0.038800,-0.005228,0.015989,-0.058992,-0.000879,0.023537,-0.025019,0.017783,-0.002662,-0.020679,0.065695
MSSubClass,0.022446,1.000000,-0.443024,-0.476489,0.054864,-0.062170,0.042091,0.059917,0.035653,-0.065747,-0.056139,-0.142379,-0.247016,-0.252038,0.314531,0.043995,0.098200,0.010519,0.004794,0.153695,0.172494,-0.041192,0.290278,0.046191,-0.031634,0.093610,-0.033940,-0.102632,-0.017182,0.012370,-0.018518,-0.038691,-0.024079,-0.008847,-0.021823,-0.007769,-0.015256,-0.057500,-0.035649
LotFrontage,-0.026217,-0.443024,1.000000,0.652855,0.225580,-0.051155,0.152837,0.081911,0.186493,0.131608,0.012585,0.158965,0.317382,0.377474,0.035450,-0.025195,0.310793,0.045918,-0.016295,0.172028,0.047932,0.268327,0.010917,0.310168,0.211539,0.088574,0.297182,0.332911,0.112689,0.110790,-0.017608,0.082919,0.060297,0.019522,0.003484,0.019379,0.025064,0.365163,0.032958
LotArea,-0.032088,-0.476489,0.652855,1.000000,0.208764,-0.025510,0.083541,0.062938,0.163373,0.155314,0.041350,0.123051,0.313049,0.404470,0.095065,-0.027804,0.384106,0.056883,0.024139,0.183185,0.101884,0.339992,0.005127,0.390368,0.261897,0.032255,0.289442,0.332412,0.160378,0.107418,-0.012648,0.060410,0.097555,0.015520,0.045803,0.012544,-0.002487,0.399529,-0.010018
OverallQual,-0.029422,0.054864,0.225580,0.208764,1.000000,-0.106079,0.590609,0.552596,0.381861,0.192706,-0.072396,0.321297,0.528592,0.458819,0.273294,-0.067799,0.580582,0.085495,-0.043012,0.555070,0.271462,0.096812,-0.187814,0.403260,0.399697,0.520893,0.598613,0.562593,0.228885,0.311074,-0.133734,0.033460,0.064631,-0.001142,-0.019989,0.073729,-0.016517,0.820706,-0.041588
OverallCond,0.015598,-0.062170,-0.051155,-0.025510,-0.106079,1.000000,-0.372541,0.067245,-0.129923,-0.036687,0.048086,-0.151546,-0.187799,-0.168622,0.021364,0.022610,-0.104415,-0.046243,0.121869,-0.218193,-0.061427,0.015616,-0.080103,-0.073074,-0.036691,-0.309961,-0.190322,-0.145101,-0.008317,-0.042501,0.084534,0.000909,0.045969,0.014921,0.086523,-0.009567,0.028408,-0.049468,0.001640
YearBuilt,-0.030417,0.042091,0.152837,0.083541,0.590609,-0.372541,1.000000,0.599308,0.308581,0.253382,-0.043821,0.155366,0.418617,0.307073,0.009709,-0.164449,0.222032,0.190404,-0.030624,0.498027,0.243079,-0.067581,-0.183746,0.099465,0.168833,0.779029,0.559815,0.510557,0.231026,0.203638,-0.403287,0.033318,-0.032414,0.005597,-0.039364,0.016340,-0.004190,0.626571,-0.051397
YearRemodAdd,-0.032972,0.059917,0.081911,0.062938,0.552596,0.067245,0.599308,1.000000,0.162907,0.116175,-0.063461,0.186589,0.303345,0.247041,0.137453,-0.077161,0.300056,0.118715,-0.008774,0.453796,0.182248,-0.043833,-0.152646,0.182423,0.113927,0.619440,0.428338,0.390195,0.216612,0.234362,-0.198103,0.032837,-0.031116,0.007416,-0.005828,0.020087,0.040943,0.589357,-0.016655
MasVnrArea,-0.057110,0.035653,0.186493,0.163373,0.381861,-0.129923,0.308581,0.162907,1.000000,0.217019,-0.068082,0.109291,0.320724,0.309999,0.147633,-0.065552,0.357217,0.066317,0.009439,0.258861,0.199513,0.095772,-0.032032,0.268199,0.259081,0.235568,0.354480,0.353953,0.137740,0.102481,-0.124251,0.020129,0.088474,-0.010533,-0.028247,0.014128,-0.003511,0.411978,-0.010135
BsmtFinSF1,-0.010659,-0.065747,0.131608,0.155314,0.192706,-0.036687,0.253382,0.116175,0.217019,1.000000,-0.053922,-0.531647,0.429338,0.355855,-0.188312,-0.072705,0.089607,0.649063,0.073085,0.034437,-0.008310,-0.1

In [109]:
## Importanto as bibliotecas usadas para o modelo de regressão linear

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.linear_model import LinearRegression

## Pegando os dados que tem relação linear com a preço da casa
df = pd.DataFrame(data, columns=['LotArea','LotFrontage','OverallQual','YearBuilt','YearRemodAdd','GarageCars','GarageArea','WoodDeckSF','OpenPorchSF','Fireplaces','TotRmsAbvGrd','FullBath','GrLivArea','1stFlrSF','TotalBsmtSF']) ** 2

x = df   
y = data['SalePrice']


In [110]:
## treinando o modelo
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)
## Regressão linear
lr = LinearRegression()
lr.fit(x_train, y_train)
y_pred = lr.predict (x_test )

In [111]:
# Valor de R2 perto de 1 nos diz que é um bom modelo
print(f"R2 score: {r2_score(y_test, y_pred)}")
# MSE Score perto de 0 é um bom modelo
print(f"MSE score: {mean_squared_error(y_test, y_pred)}")

R2 score: 0.8458653226738285
MSE score: 0.00013433246100872775


In [112]:
## Analisando a predição com o valor real
data['Pred'] = pd.DataFrame(y_pred)
data.head(100)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Pred
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,2.583824,2.570479
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,2.573300,2.563338
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,2.589054,2.546666
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,2.553297,2.539141
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,2.597433,2.605086
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,Wood,Gd,TA,No,GLQ,732,Unf,0,64,796,GasA,...,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,NaN,Attchd,1993.0,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,2.554946,2.567161
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186.0,Gd,TA,PConc,Ex,TA,Av,GLQ,1369,Unf,0,317,1686,GasA,...,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2004.0,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,2.612611,2.549118
7,8,60,RL,69.0,10382,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240.0,TA,TA,CBlock,Gd,TA,Mn,ALQ,859,BLQ,32,216,1107,GasA,...,SBrkr,1107,983,0,2090,1,0,2,1,3,1,TA,7,Typ,2,TA,Attchd,1973.0,RFn,2,484,TA,TA,Y,235,204,228,0,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal,2.580677,2.622595
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,952,952,GasA,...,FuseF,1022,752,0,1774,0,0,2,0,2,2,TA,8,Min1,2,TA,Detchd,1931.0,Unf,2,468,Fa,TA,Y,90,0,205,0,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,2.547453,2.540493
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,C